<a href="https://colab.research.google.com/github/PrestoshTJ/ML-Projects/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip


import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import cv2

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-12-03 01:14:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.207, 173.194.194.207, 142.250.152.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   160MB/s    in 0.4s    

2024-12-03 01:14:15 (160 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [6]:
cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")
i = 0
x_train = []
y_train = []

cat_files = os.listdir(cats_dir)
dog_files = os.listdir(dogs_dir)

while i < len(cat_files):
  im = Image.open(os.path.join(cats_dir, cat_files[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_train.append(np.array(im_resized))
  y_train.append(1)

  im = Image.open(os.path.join(dogs_dir, dog_files[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_train.append(np.array(im_resized))
  y_train.append(0)

  i += 1

x_train = np.array(x_train, dtype="float32")
y_train = np.array(y_train, dtype="int32")

cats_dir_test = os.path.join(validation_dir + "/cats")
dogs_dir_test = os.path.join(validation_dir + "/dogs")
i = 0
x_test = []
y_test = []

cat_files_test = os.listdir(cats_dir_test)
dog_files_test = os.listdir(dogs_dir_test)

while i < len(cat_files_test):
  im = Image.open(os.path.join(cats_dir_test, cat_files_test[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_test.append(np.array(im_resized))
  y_test.append(1)

  im = Image.open(os.path.join(dogs_dir_test, dog_files_test[i])).convert("RGB")
  im_resized = im.resize((150,150))
  x_test.append(np.array(im_resized))
  y_test.append(0)
  i += 1

x_test = np.array(x_test, dtype="float32")
y_test = np.array(y_test, dtype="int32")

In [7]:
print(f"x_train shape: {x_train.shape}  x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape} y_test shape: {y_test.shape}")

x_train shape: (2000, 150, 150, 3)  x_test shape: (1000, 150, 150, 3)
y_train shape: (2000,) y_test shape: (1000,)


In [8]:
resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

for layer in pretrained_model.layers:
  layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))
resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
resnet_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,637,826 (93.99 MB)

 Trainable params: 1,050,114 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [10]:
history = resnet_model.fit(x_train, y_train, epochs=4, validation_data=(x_test, y_test))

Epoch 1/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 324s 5s/step - accuracy: 0.7986 - loss: 1.2200 - val_accuracy: 0.9640 - val_loss: 0.0998
Epoch 2/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 254s 4s/step - accuracy: 0.9704 - loss: 0.0770 - val_accuracy: 0.9640 - val_loss: 0.1031
Epoch 3/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 258s 4s/step - accuracy: 0.9884 - loss: 0.0350 - val_accuracy: 0.9360 - val_loss: 0.1727
Epoch 4/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 262s 4s/step - accuracy: 0.9929 - loss: 0.0267 - val_accuracy: 0.9590 - val_loss: 0.1168


In [12]:
def predict_image(image_path):
  image = cv2.imread(image_path)
  image = cv2.resize(image, (150, 150))
  image = np.array(image, dtype="float32")
  image = np.expand_dims(image, axis=0)
  prediction = resnet_model.predict(image)
  print(prediction)
  if prediction[0][0] < prediction[0][1]:
    print("The predicted class is cat")
  else:
    print("The predicted class is dog")
predict_image(os.path.join(cats_dir_test, cat_files_test[0]))
predict_image(os.path.join(dogs_dir_test, dog_files_test[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[8.6134213e-07 9.9999917e-01]]
The predicted class is cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
[[1.000000e+00 7.474802e-11]]
The predicted class is dog
